In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#display everything 
pd.set_option('display.max_columns', None)

In [3]:
train = pd.read_csv('train.csv', encoding='utf-8')
test = pd.read_csv('test.csv', encoding='utf-8')
submission = pd.read_csv('sample_submission.csv')

In [4]:
train.shape, test.shape, submission.shape

((54273, 13), (36183, 12), (36183, 2))

In [5]:
display(train.isna().sum())
display(test.isna().sum())

id              0
brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
price           0
dtype: int64

id              0
brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
dtype: int64

From the above, we can see that there are no missing values in both the training and testing datasets

### Working on the engine variable

In [6]:
import re
def extract_horsepower(engine_description):
  match = re.search(r"(\d+)\.0HP", engine_description)
  if match:
    return float(match.group(1))  
  else:
    return None  # Handle missing values

train["horsepower"] = train["engine"].apply(extract_horsepower)
test["horsepower"] = test["engine"].apply(extract_horsepower)


In [7]:
import re

def extract_displacement(engine_description):
    match = re.search(r"(\d+\.\d+L)", engine_description)
    if match:
        # Extract the numeric part and convert it to a float
        return float(match.group(1).replace('L', ''))
    else:
        return None  # Handle missing values

# Assuming 'train' is your DataFrame and 'engine' is the column containing engine descriptions
train["displacement_value"] = train["engine"].apply(extract_displacement)
test["displacement_value"] = test["engine"].apply(extract_displacement)

In [8]:

def extract_engine_type(engine_description):
    # Regex to match engine type after displacement and before fuel type
    match = re.search(r"\b\d\.\dL\s(.*?Engine)\b", engine_description)
    if match:
        return match.group(1)
    else:
        return None  # Handle missing values
train["engine_type"] = train["engine"].apply(extract_engine_type)
test["engine_type"] = test["engine"].apply(extract_engine_type)

In [9]:

def extract_fuel_type(engine_description):
    # Regex to match the fuel type at the end of the description
    match = re.search(r"(Gasoline Fuel|Diesel Fuel|Electric Fuel System|Hybrid|Gasoline/Mild Electric Hybrid|Flex Fuel Capability)", engine_description)
    if match:
        return match.group(1)
    else:
        return None  # Handle missing values
train["fuel_type"] = train["engine"].apply(extract_fuel_type)
test["fuel_type"] = test["engine"].apply(extract_fuel_type)

In [10]:
# Grouping the car brands
brand_categories = {
    'performance_sports': [
        'Chevrolet', 'BMW', 'Porsche', 'Lamborghini',
        'Pontiac', 'Lotus'
    ],
    
    'luxury_luxury_EV': [
        'Mercedes-Benz', 'Audi', 'Maserati', 'Cadillac',
        'Volvo', 'Tesla', 'Jaguar', 'Rolls-Royce', 'Aston',
        'Ferrari', 'Bentley', 'Acura', 'Lexus', 'Lincoln',
        'Hummer', 'Genesis', 'Land', 'Buick', 'INFINITI',
        'McLaren', 'Alfa', 'Lucid', 'Maybach', 'Bugatti'
    ],

    'mid_range': [
        'Mitsubishi', 'Ford', 'Nissan', 'Toyota', 'Dodge',
        'Volkswagen', 'Mazda', 'Chrysler', 'Rivian', 'GMC'
    ],

    'affordable': [
        'Honda', 'Hyundai', 'Subaru', 'Kia',
        'Scion', 'Saturn', 'Mercury',
        'FIAT', 'Plymouth', 'Suzuki'
    ],

    'offroad': [
        'Jeep', 'RAM', 'MINI'
    ]
}

def assign_brand_group(brand):
    for group_name, brands in brand_categories.items():
        if brand in brands:
            return group_name
    return "Others"  # Assign to "Others" if not found in any defined groups

train["brand_group"] = train["brand"].apply(assign_brand_group)
test["brand_group"] = test["brand"].apply(assign_brand_group)


### Handling transmission variable

In [11]:
# Define the mapping for each category
transmission_mapping = {
    'automatic': ['A/T', 'Automatic', 'Automatic CVT', 'Transmission w/Dual Shift Mode', 'Transmission Overdrive Switch',
                  'Electronically Controlled Automatic', 'with Overdrive', 'with Auto-Shift', 'DCT Automatic'],
    'manual': ['M/T', 'Manual'],
    'cvt': ['CVT Transmission', 'CVT-F', 'Variable'],
    'dct': ['DCT Automatic'],
    'other': ['SCHEDULED FOR OR IN PRODUCTION', '–', 'F']
}

# Function to categorize the transmission
def categorize_transmission(transmission):
    transmission = transmission.lower()
    if any(keyword.lower() in transmission for keyword in transmission_mapping['automatic']):
        return 'automatic'
    elif any(keyword.lower() in transmission for keyword in transmission_mapping['manual']):
        return 'manual'
    elif any(keyword.lower() in transmission for keyword in transmission_mapping['cvt']):
        return 'cvt'
    elif any(keyword.lower() in transmission for keyword in transmission_mapping['dct']):
        return 'dct'
    else:
        return 'other'

# Apply the categorization
train['transmission_category'] = train['transmission'].apply(categorize_transmission)
test['transmission_category'] = test['transmission'].apply(categorize_transmission)

In [12]:
# Create boolean features for special characteristics on the training set  
train['dual_shift_mode'] = train['transmission'].str.contains('Dual Shift Mode', case=False, na=False).astype(int)
train['overdrive'] = train['transmission'].str.contains('Overdrive', case=False, na=False).astype(int)
train['auto_shift'] = train['transmission'].str.contains('Auto-Shift', case=False, na=False).astype(int)

# Create boolean features for special characteristics on the testing set
test['dual_shift_mode'] = test['transmission'].str.contains('Dual Shift Mode', case=False, na=False).astype(int)
test['overdrive'] = test['transmission'].str.contains('Overdrive', case=False, na=False).astype(int)
test['auto_shift'] = test['transmission'].str.contains('Auto-Shift', case=False, na=False).astype(int)


In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54273 entries, 0 to 54272
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     54273 non-null  int64  
 1   brand                  54273 non-null  object 
 2   model                  54273 non-null  object 
 3   model_year             54273 non-null  int64  
 4   milage                 54273 non-null  int64  
 5   fuel_type              50120 non-null  object 
 6   engine                 54273 non-null  object 
 7   transmission           54273 non-null  object 
 8   ext_col                54273 non-null  object 
 9   int_col                54273 non-null  object 
 10  accident               54273 non-null  object 
 11  clean_title            54273 non-null  object 
 12  price                  54273 non-null  int64  
 13  horsepower             50216 non-null  float64
 14  displacement_value     53667 non-null  float64
 15  en

In [14]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36183 entries, 0 to 36182
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     36183 non-null  int64  
 1   brand                  36183 non-null  object 
 2   model                  36183 non-null  object 
 3   model_year             36183 non-null  int64  
 4   milage                 36183 non-null  int64  
 5   fuel_type              33485 non-null  object 
 6   engine                 36183 non-null  object 
 7   transmission           36183 non-null  object 
 8   ext_col                36183 non-null  object 
 9   int_col                36183 non-null  object 
 10  accident               36183 non-null  object 
 11  clean_title            36183 non-null  object 
 12  horsepower             33577 non-null  float64
 13  displacement_value     35778 non-null  float64
 14  engine_type            33451 non-null  object 
 15  br

In [15]:
cols_drop = ['brand', 'model', 'engine','clean_title', 'id', 'transmission']

In [16]:
train.drop(columns=cols_drop, axis=1, inplace=True)
test.drop(columns=cols_drop, axis=1, inplace=True)

In [17]:
display(train.info())
display(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54273 entries, 0 to 54272
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   model_year             54273 non-null  int64  
 1   milage                 54273 non-null  int64  
 2   fuel_type              50120 non-null  object 
 3   ext_col                54273 non-null  object 
 4   int_col                54273 non-null  object 
 5   accident               54273 non-null  object 
 6   price                  54273 non-null  int64  
 7   horsepower             50216 non-null  float64
 8   displacement_value     53667 non-null  float64
 9   engine_type            50065 non-null  object 
 10  brand_group            54273 non-null  object 
 11  transmission_category  54273 non-null  object 
 12  dual_shift_mode        54273 non-null  int32  
 13  overdrive              54273 non-null  int32  
 14  auto_shift             54273 non-null  int32  
dtypes:

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36183 entries, 0 to 36182
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   model_year             36183 non-null  int64  
 1   milage                 36183 non-null  int64  
 2   fuel_type              33485 non-null  object 
 3   ext_col                36183 non-null  object 
 4   int_col                36183 non-null  object 
 5   accident               36183 non-null  object 
 6   horsepower             33577 non-null  float64
 7   displacement_value     35778 non-null  float64
 8   engine_type            33451 non-null  object 
 9   brand_group            36183 non-null  object 
 10  transmission_category  36183 non-null  object 
 11  dual_shift_mode        36183 non-null  int32  
 12  overdrive              36183 non-null  int32  
 13  auto_shift             36183 non-null  int32  
dtypes: float64(2), int32(3), int64(2), object(7)
memory us

None

### Imputing the missing values in both the test and train sets 

In [18]:
from sklearn.impute import KNNImputer

# Imputing categorical variables 
train['fuel_type'] = train['fuel_type'].ffill().bfill()
test['fuel_type'] =  test['fuel_type'].ffill().bfill()
train['engine_type'] = train['engine_type'].bfill().ffill()
test['engine_type'] = test['engine_type'].bfill().ffill()


In [19]:
# imputing the numerical variables 
train['horsepower'] = train['horsepower'].fillna(train['horsepower'].mean())
test['horsepower'] = test['horsepower'].fillna(test['horsepower'].median())
train['displacement_value'] = train['displacement_value'].fillna(train['displacement_value'].mean())
test['displacement_value'] = test['displacement_value'].fillna(test['displacement_value'].median())

### Encoding the categorical variables 

In [20]:
## Encoding training data
from sklearn.preprocessing import LabelEncoder
Le_fuel = LabelEncoder()
Le_ext = LabelEncoder()
Le_int = LabelEncoder()
Le_acc = LabelEncoder()
Le_engine = LabelEncoder()
Le_brand = LabelEncoder()
Le_trans_cat = LabelEncoder()

train['ext_col'] = Le_ext.fit_transform(train['ext_col'])
train['int_col'] = Le_int.fit_transform(train['int_col'])
train['accident'] = Le_acc.fit_transform(train['accident'])
train['engine_type'] = Le_engine.fit_transform(train['engine_type'])
train['brand_group'] = Le_brand.fit_transform(train['brand_group'])
train['transmission_category'] = Le_trans_cat.fit_transform (train['transmission_category'])
train['fuel_type'] = Le_fuel.fit_transform(train['fuel_type'])

In [21]:
#Encoding testing data

test['ext_col'] = Le_ext.fit_transform(test['ext_col'])
test['int_col'] = Le_int.fit_transform(test['int_col'])
test['accident'] = Le_acc.fit_transform(test['accident'])
test['engine_type'] = Le_engine.fit_transform(test['engine_type'])
test['brand_group'] = Le_brand.fit_transform(test['brand_group'])
test['transmission_category'] = Le_trans_cat.fit_transform(test['transmission_category'])
test['fuel_type'] = Le_fuel.fit_transform(test['fuel_type'])

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor

In [23]:
X = train.drop(columns='price', axis=1, inplace=False) 
y = train['price']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.2, random_state=42 )


In [25]:

#Scaling all columns using MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)

#transforming the testing adn training data with the scaler
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

#converting the scaled arrays to dataframes
X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns = X_test.columns)

In [42]:
from sklearn.tree import DecisionTreeRegressor
DT = DecisionTreeRegressor(random_state=42)

In [43]:

# Train the model
DT.fit(X_train_scaled, y_train)

# Make predictions on the test set
predictions = DT.predict(X_test_scaled)

# Calculate RMSE
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"Decision Tree Regressor RMSE: {rmse}")

Decision Tree Regressor RMSE: 142614.59283761005


c:\Users\PC\anaconda3-2\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [40]:
# Create submission file
#sub1 = pd.DataFrame({'id': submission['id'], 'price': boost_predictions})

# Preview sub file
#sub1.head()

,id,price
0,54273,256577.493145
1,54274,256577.493145
2,54275,256577.493145
3,54276,194143.479101
4,54277,205767.488287


In [41]:
# Create a csv file
#sub1.to_csv('submission8.csv', index = False)